<a href="https://colab.research.google.com/github/LongNguyen1984/GANLearning/blob/main/Ass1MLOPsCourse2_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install the dataset

In [1]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00296/dataset_diabetes.zip
!unzip dataset_diabetes.zip &> 0


--2021-09-27 13:58:17--  https://archive.ics.uci.edu/ml/machine-learning-databases/00296/dataset_diabetes.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3347213 (3.2M) [application/x-httpd-php]
Saving to: ‘dataset_diabetes.zip’

dataset_diabetes.zi 100%[===================>]   3.19M  6.14MB/s    in 0.5s    

2021-09-27 13:58:18 (6.14 MB/s) - ‘dataset_diabetes.zip’ saved [3347213/3347213]



### Get the file for assignment

In [3]:
!wget https://github.com/amanchadha/coursera-machine-learning-engineering-for-prod-mlops-specialization/blob/main/C2%20-%20Machine%20Learning%20Data%20Lifecycle%20in%20Production/Week%201/util.py

--2021-09-27 14:03:08--  https://github.com/amanchadha/coursera-machine-learning-engineering-for-prod-mlops-specialization/blob/main/C2%20-%20Machine%20Learning%20Data%20Lifecycle%20in%20Production/Week%201/util.py
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘util.py’

util.py                 [ <=>                ] 161.93K  --.-KB/s    in 0.04s   

2021-09-27 14:03:08 (4.22 MB/s) - ‘util.py’ saved [165815]



In [2]:
!pip install tensorflow-data-validation

     |████████████████████████████████| 1.4 MB 5.1 MB/s 
     |████████████████████████████████| 9.8 MB 35.9 MB/s 
     |████████████████████████████████| 294 kB 71.9 MB/s 
     |████████████████████████████████| 17.7 MB 79 kB/s 
     |████████████████████████████████| 19.0 MB 89 kB/s 
     |████████████████████████████████| 829 kB 65.9 MB/s 
     |████████████████████████████████| 62 kB 852 kB/s 
     |████████████████████████████████| 234 kB 19.8 MB/s 
     |████████████████████████████████| 2.3 MB 34.9 MB/s 
     |████████████████████████████████| 151 kB 76.4 MB/s 
     |████████████████████████████████| 173 kB 67.8 MB/s 
     |████████████████████████████████| 435 kB 66.5 MB/s 
     |████████████████████████████████| 144 kB 73.0 MB/s 
     |████████████████████████████████| 169 kB 75.3 MB/s 
     |████████████████████████████████| 83 kB 2.2 MB/s 
     |████████████████████████████████| 267 kB 70.8 MB/s 
     |████████████████████████████████| 180 kB 72.4 MB/s 
     |███████████████

In [1]:
# Import packages
import os
import pandas as pd
import tensorflow as tf
import tempfile, urllib, zipfile
import tensorflow_data_validation as tfdv


from tensorflow.python.lib.io import file_io
from tensorflow_data_validation.utils import slicing_util
from tensorflow_metadata.proto.v0.statistics_pb2 import DatasetFeatureStatisticsList, DatasetFeatureStatistics

# Set TF's logger to only display errors to avoid internal warnings being shown
tf.get_logger().setLevel('ERROR')

### Load the dataset

In [6]:
# Read the dataset and recognize the missing data that in encoded with '?' as Nan

df = pd.read_csv('dataset_diabetes/diabetic_data.csv', header=0, na_values ='?')

# preview the dataset
df.info()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 50 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   encounter_id              101766 non-null  int64 
 1   patient_nbr               101766 non-null  int64 
 2   race                      99493 non-null   object
 3   gender                    101766 non-null  object
 4   age                       101766 non-null  object
 5   weight                    3197 non-null    object
 6   admission_type_id         101766 non-null  int64 
 7   discharge_disposition_id  101766 non-null  int64 
 8   admission_source_id       101766 non-null  int64 
 9   time_in_hospital          101766 non-null  int64 
 10  payer_code                61510 non-null   object
 11  medical_specialty         51817 non-null   object
 12  num_lab_procedures        101766 non-null  int64 
 13  num_procedures            101766 non-null  int64 
 14  num_

### Data split
will split the dataset into:

* 70% training set
* 15% evaluation set
* 15% serving set


In [11]:
def prepare_data_splits_from_dataframe(df):
  '''
    Splits a Pandas Dataframe into training, evaluation and serving sets.

    Parameters:
            df : pandas dataframe to split

    Returns:
            train_df: Training dataframe(70% of the entire dataset)
            eval_df: Evaluation dataframe (15% of the entire dataset) 
            serving_df: Serving dataframe (15% of the entire dataset, label column dropped)
    '''
    # 70% of records for generating the training set
  train_len = int(len(df)*0.7)

    # Remaining 30% of records for generating the evaluation and serving sets
  eval_serv_len = len(df) - train_len

  eval_len = eval_serv_len//2

  ser_len = eval_serv_len - eval_len

    # Sample the train, validation and serving set.
  train_df = df.iloc[:train_len].sample(frac=1, random_state=42).reset_index(drop=True)
  eval_df = df.iloc[train_len:train_len + eval_len].sample(frac=1, random_state=42).reset_index(drop=True)
  ser_df = df.iloc[train_len + eval_len: train_len + eval_len + ser_len].sample(frac=1, random_state=42).reset_index(drop=True)

    # Serving data emuates the data that would be submitted for predictions so it not have the label column.
  ser_df = ser_df.drop(['readmitted'], axis=1)

  return train_df, eval_df, ser_df
    


In [13]:
## Split the datasset

train_df, eval_df, ser_df = prepare_data_splits_from_dataframe(df)
print('Training dataset has {} records\n Validation dataset has {} records\nServing dataset has {} records'.format(len(train_df), len(eval_df), len(ser_df)))

Training dataset has 71236 records
 Validation dataset has 15265 records
Serving dataset has 15265 records
